In [29]:
# Ticket data
import pandas as pd

df = pd.read_excel("../data/JIRA-Export-13.10.2022.xlsx", engine="openpyxl")
df.head();

In [30]:
df = df[["Beschreibung", "Benötigte Zeit"]]
df = df.rename(columns={"Beschreibung":"x", "Benötigte Zeit":"y"})
df["y"] = df["y"].astype(str)
df["x"] = df["x"].astype(str)
df;

In [27]:
# Load Base Model
from transformers import AutoTokenizer

BASE_MODEL = "bert-base-german-cased"
#BASE_MODEL = "bert-base-cased"
LEARNING_RATE = 5e-5
MAX_LENGTH = 256
BATCH_SIZE = 32 # perhaps use 8 to 16
EPOCHS = 4 # 20 -> use stop condition

#id2label = #TODO: Define {0,1,2,.. : 1h,2h,4h,..} 
#label2id = #TODO: inverted id2label

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)


loading configuration file https://huggingface.co/bert-base-german-cased/resolve/main/config.json from cache at C:\Users\User/.cache\huggingface\transformers\98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e
Model config BertConfig {
  "_name_or_path": "bert-base-german-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading file https://huggingface.co/bert-base-german-cased/resolve/main/vocab.txt

In [28]:
from transformers import AutoModelForSequenceClassification

# classification:
#model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)

# regression:
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)

loading configuration file https://huggingface.co/bert-base-german-cased/resolve/main/config.json from cache at C:\Users\User/.cache\huggingface\transformers\98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e
Model config BertConfig {
  "_name_or_path": "bert-base-german-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loa

In [31]:
# Tokenize input
def token_preprocessor(df):
    label = df["y"] 
    df = tokenizer(df["x"], padding="max_length", max_length=256, truncation=True)
    # classification:
    # df["label"] = label
    # regression
    df["label"] = float(label)
    #print(df)
    return df
    #return tokenizer(df["x"], padding="max_length", max_length=256, truncation=True)

In [32]:
# Prepare Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset

train, test = train_test_split(df, test_size=0.20, random_state=123)
#X_train, X_test, y_train, y_test = train_test_split(df["X"], df["y"], test_size=0.20, random_state=123)

ds = {"train": Dataset.from_dict(train), "test": Dataset.from_dict(test)}

In [33]:
for split in ds:
    ds[split] = ds[split].map(token_preprocessor, remove_columns=["x", "y"])

  0%|          | 0/3543 [00:00<?, ?ex/s]

  0%|          | 0/886 [00:00<?, ?ex/s]

In [15]:
ds

{'train': Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 3543
 }),
 'test': Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 886
 })}

In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()
    
    # Compute accuracy 
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
    
    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [34]:
# Training Params
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../trainoutput/",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [35]:
import torch
from transformers import Trainer

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss
    
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=compute_metrics_for_regression
)

trainer.train()

C:\Users\User\anaconda3\envs\Huggingface36\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3543
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 444


Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,No log,4102520576.000000,4102520576.000000,34813.574219,-0.419294,0.000000
2,No log,4102312704.000000,4102313472.000000,34810.601562,-0.419223,0.000000
3,No log,4102196224.000000,4102196224.000000,34808.921875,-0.419182,0.000000
4,No log,4102157824.000000,4102157824.000000,34808.371094,-0.419169,0.000000


***** Running Evaluation *****
  Num examples = 886
  Batch size = 32
Saving model checkpoint to ../trainoutput/checkpoint-111
Configuration saved in ../trainoutput/checkpoint-111\config.json
Model weights saved in ../trainoutput/checkpoint-111\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 886
  Batch size = 32
Saving model checkpoint to ../trainoutput/checkpoint-222
Configuration saved in ../trainoutput/checkpoint-222\config.json
Model weights saved in ../trainoutput/checkpoint-222\pytorch_model.bin
Deleting older checkpoint [..\trainoutput\checkpoint-4440-20eps-2e-5-16batch] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 886
  Batch size = 32
Saving model checkpoint to ../trainoutput/checkpoint-333
Configuration saved in ../trainoutput/checkpoint-333\config.json
Model weights saved in ../trainoutput/checkpoint-333\pytorch_model.bin
Deleting older checkpoint [..\trainoutput\checkpoint-222] due to args.save_total_limit
***** Running Eva

TrainOutput(global_step=444, training_loss=3266248408.792793, metrics={'train_runtime': 19562.0509, 'train_samples_per_second': 0.724, 'train_steps_per_second': 0.023, 'total_flos': 1864388198541312.0, 'train_loss': 3266248408.792793, 'epoch': 4.0})

In [36]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
No module named 'transformers.pipelines'

In [37]:
test_string1 = "Das Icon muss ausgewechselt werden! Lorem ipsum dolor sit amet, dnmsakd \n hahah IT User, Oberfläche"
test_string2 = """IST: 
Wird im EHB "Stichprobenbildung ohne Ergebniserfassung" im Abschnitt "Stichprobenbildung vollstationär ohne Ergebniserfassung" "Regelprüfung" markiert, dann ist im Abschnitt "Ergänzende Stichprobe bei Anlass- /Wiederholungsprüfung" über den Button "Weiteres" eine weitere Person erfassbar. 

Soll: 
Im Abschnitt "Ergänzende Stichprobe bei Anlass- /Wiederholungsprüfung" soll ausschließlich eine Person erfassbar sein, wenn in Abschnitt "Stichprobenbildung vollstationär ohne Ergebniserfassung" "Anlass/Wiederholungsprüfung" markiert wurde 
"""
test_string3 = "Frage 3.5 (E Frage) wurde im EHB Einrichtung ausgefüllt und \"Empfehlung geben\" angekreuzt \nIm Produktergebnis wird Frage 3.5 nicht in die Tabelle(Empfehlungen zur Beseitigung von Qualitätsdefiziten) aufgenommen"
some_strings = [test_string1, test_string2, test_string3]

In [40]:
test

,x,y
31,IST: \nDaten in Prüfbogen A: \n* B14 = ja \n* ...,10800
1756,*edit_CB:* \n\nSiehe Kommentar aus dem MMI von...,23400
3842,Im EHB Person Kapitel 9.7 wird die Checkbox Ze...,5400
2142,.edit_MP: \n\nNach Absprache mit Michael H. zu...,5400
4087,Wird eine Aenderung einer Telefonnummer ausgef...,5400
...,...,...
3894,IST: \n* Maske Auftrag/Vorgang erstellen ist w...,19800
2291,Unsere custom lint rule [{{no-component-import...,26100
2357,+Ist:+ \nDie Schlüsselliste zur Präzisierung d...,9000
4174,"Beim einloggen z.B. kann es vorkommen, dass da...",3600


In [38]:
encoded = tokenizer(some_strings, padding="max_length", max_length=256, truncation=True, return_tensors="pt")

In [ ]:
list(test["x"][:10]

SyntaxError: unexpected EOF while parsing (<ipython-input-47-fc99c550c169>, line 1)

In [45]:
encoded = tokenizer(list(test["x"][:10]), padding="max_length", max_length=256, truncation=True, return_tensors="pt")

In [46]:
model(**encoded).logits.reshape(-1).tolist()

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 31457280 bytes.